Single & Multiple Unzip Files

In [6]:
import zipfile
import os
import json
import pandas as pd
import os
from pathlib import Path
import shutil
from tqdm import tqdm

In [7]:
# config zip -> upzip

SRC_ZIP = r"to_yolo\zip"
DST_UNZIP = r"to_yolo\unzip"

In [8]:
zip_files = os.listdir(SRC_ZIP)
print(zip_files)

for zip_ in zip_files:

    src = os.path.join(SRC_ZIP, zip_)
    folder_name = zip_[:zip_.find('.zip')]
    dst = os.path.join(DST_UNZIP, folder_name)

    with zipfile.ZipFile(src, 'r') as my_zip:
        my_zip.extractall(dst)

['Set 49.zip']


### convert COCO to YOLO # SEGMANTATION

In [9]:
# config COCO path -> YOLO path

SRC_CONVERT_RESULTS = R"to_yolo\unzip"
DST_CONVERT_RESULTS = R'to_yolo\results'

In [97]:
folders = os.listdir(SRC_CONVERT_RESULTS)
print(folders)

for folder in tqdm(folders):

    src = SRC_CONVERT_RESULTS
    dst = DST_CONVERT_RESULTS
    
    with open(os.path.join(src, folder, 'annotations', 'instances_default.json'), 'r') as json_file:
        json_data = json.load(json_file)
    images_info = pd.DataFrame(json_data['images'])
    annotations = pd.DataFrame(json_data['annotations'])
    
    os.makedirs(os.path.join(dst, folder+'_yolo'), exist_ok=True)
    os.makedirs(os.path.join(dst, folder+'_yolo', 'images'), exist_ok=True)
    os.makedirs(os.path.join(dst, folder+'_yolo', 'labels'), exist_ok=True)

    for im_info in images_info.iloc:
        anno_select = annotations[annotations['image_id'] == im_info['id']]
        
        # fix this to flter class 
        # class_select = anno_select[(anno_select['category_id'] == 1) | (anno_select['category_id'] == 2) | (anno_select['category_id'] == 3)]
        class_select = anno_select[(anno_select['category_id'] == 1) | (anno_select['category_id'] == 2)]
        
        if not class_select.empty:
            seg = class_select['segmentation']
            text = ''
            for index,s in enumerate(seg):

                # define class number 
                cls = class_select['category_id'].values[index]-1 # select all classs default
                
                sn = [val/im_info['width'] if i%2 == 0 else val/im_info['height'] for i,val in enumerate(s[0])]
                text += f'{cls} '
                text += ' '.join([f'{val}' for val in sn])
                text += '\n'
        else: text = ''
        
        with open(os.path.join(dst, folder+'_yolo', 'labels', im_info['file_name'][:-4] +'.txt'), 'w') as lb:
            lb.write(text)
        
        shutil.copy(os.path.join(src, folder, 'images', im_info['file_name']) , os.path.join(dst, folder+'_yolo', 'images',im_info['file_name']))


['Set 49']


100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


## spilt data

In [6]:
# config split path
from pathlib import Path
SRC_SPLIT = Path(r'C:\Users\JiraponSasomsap\Downloads\lime_bboxes\results')
DST_SPLIT = Path(r'C:\Users\JiraponSasomsap\Downloads\lime_bboxes\split')

FRIST_RUNNING = True

In [7]:
import random
import os
from tqdm import tqdm 


assert FRIST_RUNNING

RATIO = (0.8, 0.2, 0.0)

def split_data(items, train, val , test):
    set_names = 'train,val,test'.split(',')

    assert train + val + test == 1.0, "train, val, and test splits must add up to 1.0"
    
    train_n = round(len(items)*train)
    val_n = round(len(items)*val)
    
    train_set = items[:train_n]
    val_set = items[train_n:train_n + val_n]
    test_set = items[train_n + val_n:]

    for spt_items, set_name in zip((train_set, val_set, test_set), set_names):
        # print(set_name, spt_items)
        for item in spt_items:
            dst = Path(DST_SPLIT / ''.join([str(item.parent).split('\\')[2], '_split']) / set_name)
            os.makedirs(dst / 'images', exist_ok=True)  
            os.makedirs(dst / 'labels', exist_ok=True)
            shutil.copy(item, dst / 'images' / item.name)
            lb = [i for i in Path(SRC_SPLIT).glob(f'*/labels/{item.stem}.txt')][0]
            shutil.copy(lb, dst / 'labels' / lb.name)

for img_items in tqdm([i for i in SRC_SPLIT.glob('*/images')]):
    items = [item for item in img_items.glob('*.jpg')]
    # print(items)
    random.shuffle(items)
    # print(items)
    split_data(items, *RATIO)

FRIST_RUNNING = False

0it [00:00, ?it/s]
